In [1]:
! pip install peft bitsandbytes accelerate datasets

  Attempting uninstall: nvidia-cusolver-cu12
    Found existing installation: nvidia-cusolver-cu12 11.6.3.83
    Uninstalling nvidia-cusolver-cu12-11.6.3.83:
      Successfully uninstalled nvidia-cusolver-cu12-11.6.3.83
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2025.3.0 which is incompatible.


In [2]:
import os
from typing import List, Dict, Any

import torch
import torch.nn as nn
from torch.nn.utils.rnn import pad_sequence
import torch.optim as optim
from transformers import AutoModelForCausalLM, AutoTokenizer, Trainer, TrainingArguments, BitsAndBytesConfig, AutoConfig, set_seed
from torch.utils.data import Dataset
from datasets import load_dataset
from peft import PeftModel, get_peft_model, LoraConfig, prepare_model_for_kbit_training

set_seed(12, True)

os.environ["WANDB_DISABLED"] = "true"
os.environ["CUBLAS_WORKSPACE_CONFIG"] = ":16:8"

# Gradient Accumulation - 5 баллов

Давайте реализуем собственную аккумуляцию градиентов.
Ниже описано обучение обычного линейного слоя. Клеткой ниже этот код скопирован, там необходимо написать аккумуляцию ргадиентов.

In [3]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
input_size = 512
output_size = 256
batch_size = 64
gradient_accumulation_steps = 4



model = nn.Linear(input_size, output_size).to(device)
optimizer = optim.SGD(model.parameters(), lr=0.001)

x = torch.randn(batch_size, input_size).to(device)
y = torch.randn(batch_size, output_size).to(device)
loss_fn = nn.MSELoss()
for i in range(1000):
    optimizer.zero_grad()
    output = model(x)
    loss = loss_fn(output, y)
    loss.backward()
    optimizer.step()

print(loss.item())

1.1878371238708496


Число шагов в аккумуляции определяется параметром gradient_accumulation_steps - это число шагов, которое мы хотим сделать перед оптимизацией.
Вам нужно поправить цикл обучения следующим образом:
1. Разбить текущий батч на gradient_accumulation_steps частей
2. Пройтись по каждому подбатчу (микробатчу), посчитать на нем функцию потерь, посчитать градиенты. Подумайте, нужно ли на что-либо делить или умножать функцию потерь, чтобы сохранился тот же масштаб обучения?
3. После прохождения всех микробатчей нужно сделать шаг оптимизации

In [4]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
input_size = 512
output_size = 256
batch_size = 64
gradient_accumulation_steps = 4



model = nn.Linear(input_size, output_size).to(device)
optimizer = optim.SGD(model.parameters(), lr=0.001)

x = torch.randn(batch_size, input_size).to(device)
y = torch.randn(batch_size, output_size).to(device)
loss_fn = nn.MSELoss()
for i in range(1000):
    optimizer.zero_grad()
    chunks = x.chunk(gradient_accumulation_steps)
    chunk_loss = 0
    for chunk in chunks:
        output = model(x)
        chunk_loss += loss_fn(output, y)
    loss = chunk_loss / gradient_accumulation_steps
    loss.backward()
    optimizer.step()

print(loss.item())

1.1858601570129395


# QLORA - 15 баллов
Необходимо использовать аккумуляцию градиентов, чекпоинтинг активаций и обучение qlora.

In [5]:
model_name = "NousResearch/Llama-2-7b-hf"
tokenizer = AutoTokenizer.from_pretrained(model_name)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/746 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/435 [00:00<?, ?B/s]

In [6]:
imdb = load_dataset("imdb")

README.md:   0%|          | 0.00/7.81k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/21.0M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/20.5M [00:00<?, ?B/s]

unsupervised-00000-of-00001.parquet:   0%|          | 0.00/42.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

Наша задача научиться генерировать класс текста posive или negative, чтобы сэкономить на fewshot промпте.

Давайте напишем collate_fn, которая собирает сэмпл следующим образом:

если текст имеет метку 1
`{text} ||| posive eos`
или
`{text} ||| negatve eos`
если текст имеет метку 0. (в качестве eos можно использовать tokenizer.eos_token_id)

Символы ||| нужны нам, чтобы разделить входной текст и метку, иначе модель может не понять, что нужно генерировать метку и продолжит генерировать текст. Таким образом мы научим модель после ||| генерировать положительный или отрицательнй отзыв стоит до этого.


Возвращать нужно словарь из 3х элементов:
1. input_ids - LongTensor токенов. В качестве паддинга нужно использовать tokenizer.eos_token_id.
2. attention_mask - LongTensor той же размерности, что и input_ids. 0 там, где стоят паддинги, 1 в остальных позициях
3. labels - метки, которые мы предсказыаем. Должен быть равен -100 на всех позициях, кроме позиций, которые соответствуют метке и eos символу.
Например
```python
tokenizer.encode("some text ||| positive </s>") # [1, 777, 1426, 3830, 29989, 6374, 2]
labels = [-100, -100, -100, -100, -100, 6374, 2]
```

Т.е. метки должны быть -100, кроме позиций, соответствующих предсказываемым токенам.

In [7]:
def collate_fn(batch: List[Dict[str, Any]]) -> Dict[str, torch.Tensor]:
  """Форматируем данные для тренировки.

  Выходные данные имеют вид:
  {text} ||| positive tokenizer.eos_token_id -- если текст имеет метку 1
  {text} ||| negative tokenizer.eos_token_id -- если текст имеет метку 0

  Outputs:
    Словарь с тремя ключами.
    input_ids - LongTensor токенов. В качестве паддинга нужно использовать tokenizer.eos_token_id.
    attention_mask - LongTensor той же размерности, что и input_ids. 0 там, где стоят паддинги, 1 в остальных позициях
    labels - метки, которые мы предсказыаем. Должен быть равен -100 на всех позициях, кроме позиций, которые соответствуют метке и eos символу.
  """
  class_mapping = {0: "negative", 1: "positive"}
  texts = [sample["text"] + " ||| " + class_mapping[sample["label"]] + " </s>" for sample in batch]

  input_ids = [tokenizer.encode(el) for el in texts]
  max_len = max(len(el) for el in input_ids)
  attention_mask = [[0] * (max_len - len(el)) + [1] * len(el) for el in input_ids]
  padded_input_ids = [[tokenizer.eos_token_id] * (max_len - len(el)) + el for el in input_ids]
  labels = [[-100] * (max_len-2) + el[-2:] for el in padded_input_ids]
  return {
      "input_ids": torch.tensor(padded_input_ids).long(),
      "attention_mask": torch.tensor(attention_mask).long(),
      "labels": torch.tensor(labels).long(),
  }

res = collate_fn([imdb["train"][0], imdb["train"][12505], imdb["train"][2]])

assert tokenizer.decode(res["input_ids"][res["labels"] != -100]) == "negative</s> positive</s> negative</s>"

Далее нам нужно создать модель в nf4, т.е. 4-битной квантизации. Конфиг уже написан, нужно лишь подать его в модель. После этого нужно:
1. Создать конфиг адаптера LoraConfig (используйте r=8 или r=4, если будет OOM) и создать модель
2. Создать модель с адаптером с помощью PeftModel и LoraConfig
3. Чтобы обучение шло только по lora частям, нужно пройтись по всем параметрам модели с помощью model.named_parameters() и проставить у параметров, соответствующих lora атрибут requires_grad = True, а у всех остальных False

In [8]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_storage=torch.bfloat16,
)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    torch_dtype=torch.bfloat16,
)

model = prepare_model_for_kbit_training(model, use_gradient_checkpointing=True)

config.json:   0%|          | 0.00/583 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/200 [00:00<?, ?B/s]

In [15]:
# смотрим, как называются слои в модели, чтобы подать верные имена в LoRaConfig
for n, (name, module) in enumerate(model.named_modules()):
  print(n, name)
  if n > 15:
    break

0 
1 model
2 model.embed_tokens
3 model.layers
4 model.layers.0
5 model.layers.0.self_attn
6 model.layers.0.self_attn.q_proj
7 model.layers.0.self_attn.k_proj
8 model.layers.0.self_attn.v_proj
9 model.layers.0.self_attn.o_proj
10 model.layers.0.mlp
11 model.layers.0.mlp.gate_proj
12 model.layers.0.mlp.up_proj
13 model.layers.0.mlp.down_proj
14 model.layers.0.mlp.act_fn
15 model.layers.0.input_layernorm
16 model.layers.0.post_attention_layernorm


In [16]:
peft_config = LoraConfig(
    r=8,
    # target_modules='all-linear',
    target_modules=["q_proj", "v_proj", "k_proj"],
    # lora_dropout=.05
)
model = get_peft_model(model=model, peft_config=peft_config)
# не забудьте пройтись по всем параметрам и проставить .requires_grad там, где нужно

# ручное присваивание requires_grad не потребовалось, т.к. get_peft_model об этом позаботился

In [17]:
# проверяем, что параметры модели с градиентом есть и относятся к lora слоям
for n, (name, param) in enumerate(model.named_parameters()):
  if param.requires_grad:
    print(name)
  if n > 15:
    break

base_model.model.model.layers.0.self_attn.q_proj.lora_A.default.weight
base_model.model.model.layers.0.self_attn.q_proj.lora_B.default.weight
base_model.model.model.layers.0.self_attn.k_proj.lora_A.default.weight
base_model.model.model.layers.0.self_attn.k_proj.lora_B.default.weight
base_model.model.model.layers.0.self_attn.v_proj.lora_A.default.weight
base_model.model.model.layers.0.self_attn.v_proj.lora_B.default.weight


In [ ]:
# ожидаемо видим, что обучаемых параметров не так много, т.е. это только слои lora
model.print_trainable_parameters()

trainable params: 6,291,456 || all params: 6,744,707,072 || trainable%: 0.0933


Осталось самое важное, аргументы обучения. Обязательно заполните следующие параметры:

1. Батч сайз и число шагов аккумуляции выставьте так, чтобы эффективный батч сайз был 16
2. Включите чекпоинтинг активаций

In [ ]:
args = TrainingArguments(
    gradient_checkpointing=True,
    # эффективный батч сайз = gradient_accumulation_steps * per_device_train_batch_size
    gradient_accumulation_steps=4,
    per_device_train_batch_size=4,
    report_to=None,
    remove_unused_columns=False,
    label_names=["labels"],
    # ограничиваем кол-во шагов, как советовали в чате; берем с запасом
    max_steps=100
)

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=imdb["train"],
    tokenizer=tokenizer,
    data_collator=collate_fn,
)
trainer.train()

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
<ipython-input-15-3f6bf67b9443>:12: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.


Step,Training Loss


Step,Training Loss


TrainOutput(global_step=100, training_loss=2.3372998046875, metrics={'train_runtime': 12348.5016, 'train_samples_per_second': 0.13, 'train_steps_per_second': 0.008, 'total_flos': 4.052909644834406e+16, 'train_loss': 2.3372998046875, 'epoch': 0.064})

Давайте протестируем, что модель что-то выучила

In [ ]:
input_text = imdb["test"][0]["text"] + " |||"
label = imdb["test"][0]["label"]
x = tokenizer(input_text, return_tensors="pt")
for k, v in x.items():
    x[k] = v.cuda()

print(label)
g = model.generate(**x, max_new_tokens=2, do_sample=False)
print(tokenizer.decode(g[0].tolist()))

0


/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:631: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:636: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/utils/checkpoint.py:87: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(


<s> I love sci-fi and am willing to put up with a lot. Sci-fi movies/TV are usually underfunded, under-appreciated and misunderstood. I tried to like this, I really did, but it is to good TV sci-fi as Babylon 5 is to Star Trek (the original). Silly prosthetics, cheap cardboard sets, stilted dialogues, CG that doesn't match the background, and painfully one-dimensional characters cannot be overcome with a 'sci-fi' setting. (I'm sure there are those of you out there who think Babylon 5 is good sci-fi TV. It's not. It's clichéd and uninspiring.) While US viewers might like emotion and character development, sci-fi is a genre that does not take itself seriously (cf. Star Trek). It may treat important issues, yet not as a serious philosophy. It's really difficult to care about the characters here as they are not simply foolish, just missing a spark of life. Their actions and reactions are wooden and predictable, often painful to watch. The makers of Earth KNOW it's rubbish as they have to a

In [ ]:
# <s> I love sci-fi and am willing to put up with a lot.
# Sci-fi movies/TV are usually underfunded, under-appreciated and misunderstood.
# I tried to like this, I really did, but it is to good TV sci-fi as Babylon 5 is to Star Trek (the original).
# Silly prosthetics, cheap cardboard sets, stilted dialogues,
# CG that doesn't match the background, and painfully one-dimensional characters cannot be overcome with
# a 'sci-fi' setting. (I'm sure there are those of you out there who think Babylon 5 is good sci-fi TV.
# It's not. It's clichéd and uninspiring.) While US viewers might like emotion and character development,
# sci-fi is a genre that does not take itself seriously (cf. Star Trek).
# It may treat important issues, yet not as a serious philosophy.
# It's really difficult to care about the characters here as they are not simply foolish,
#  just missing a spark of life. Their actions and reactions are wooden and predictable,
#  often painful to watch. The makers of Earth KNOW it's rubbish as they have to always say
#  "Gene Roddenberry's Earth..." otherwise people would not continue watching. Roddenberry's
#  ashes must be turning in their orbit as this dull, cheap, poorly edited (watching it without
# advert breaks really brings this home) trudging Trabant of a show lumbers into space. Spoiler.
# So, kill off a main character. And then bring him back as another actor. Jeeez! Dallas all over again.
# ||| negative</s>
